How can we efficiently produce ground states of a quadratic Hamiltonian but not other Hamiltonians?

In [21]:
from openfermion.hamiltonians import fermi_hubbard 
hub = fermi_hubbard(2, 2, 1., 2.) # This is a FermionOperator instance

`hub` is a `FermionOperator` instance now, but we want to get it in the form: 
$$ H = \sum_{pq} T_{pq} a^\dagger_p a_q + \sum_{pq} V_{pq} a^\dagger_p a_p a^\dagger_q a_q + constant $$

This is called a diagonal coulomb operator because our coulomb term is only a function of 2 indices instead of the usual four. The Hubbard Hamiltonian can be diagonalized like that, so we can call `get_diagonal_coulomb_operator()` to get that representation. 

In [22]:
from openfermion.transforms import get_diagonal_coulomb_hamiltonian
# print("One-body term: ", dc_hub.one_body)
# print("Two-body term: ", dc_hub.two_body)
# print("Constant term: ", dc_hub.constant)

We're interested in getting the ground state of the one-body term. Because it's a quadratic Hamiltonian, we can change basis (by applying a Bogoliubov transformation) to diagonalize it in the form $$\sum_p \epsilon_p b_p^\dagger b_p$$ where $b_p$ is a linear combination of $a_p$.

In [23]:
from openfermion.ops import QuadraticHamiltonian
quad_hub = QuadraticHamiltonian(dc_hub.one_body)
quad_hub.diagonalizing_bogoliubov_transform()

(array([-4., -4., -1., -1., -1., -1., -1., -1., -1., -1.,  2.,  2.,  2.,
         2.,  2.,  2.,  2.,  2.]),
 array([[ 3.33333333e-01+0.j,  0.00000000e+00+0.j,  3.33333333e-01+0.j,
          0.00000000e+00+0.j,  3.33333333e-01+0.j,  0.00000000e+00+0.j,
          3.33333333e-01+0.j,  0.00000000e+00+0.j,  3.33333333e-01+0.j,
          0.00000000e+00+0.j,  3.33333333e-01+0.j,  0.00000000e+00+0.j,
          3.33333333e-01+0.j,  0.00000000e+00+0.j,  3.33333333e-01+0.j,
          0.00000000e+00+0.j,  3.33333333e-01+0.j,  0.00000000e+00+0.j],
        [ 0.00000000e+00+0.j, -3.33333333e-01+0.j,  0.00000000e+00+0.j,
         -3.33333333e-01+0.j, -2.22044605e-16+0.j, -3.33333333e-01+0.j,
          1.11022302e-16+0.j, -3.33333333e-01+0.j,  5.55111512e-17+0.j,
         -3.33333333e-01+0.j, -2.77555756e-17+0.j, -3.33333333e-01+0.j,
          5.55111512e-17+0.j, -3.33333333e-01+0.j,  4.16333634e-17+0.j,
         -3.33333333e-01+0.j, -2.77555756e-17+0.j, -3.33333333e-01+0.j],
        [ 0.00000000e+00+0

In [28]:
from openfermioncirq.primitives import bogoliubov_transform
# bogoliubov_transform??
from openfermion.utils import count_qubits
import cirq

# Get Bogoliubov transformation matrix
_, transformation_matrix, _ = quad_hub.diagonalizing_bogoliubov_transform()

# Create circuit that prepares the mean-field state 
n_electrons = 2
occupied_orbitals = range(n_electrons)
n_qubits = count_qubits(quad_hub)
qubits = cirq.LineQubit.range(n_qubits)
state_preparation_circuit = cirq.Circuit(
    bogoliubov_transform(qubits, transformation_matrix, initial_state=occupied_orbitals))

# Print circuit 
cirq.DropNegligible().optimize_circuit(state_preparation_circuit)
print(state_preparation_circuit)

0: ───────────────────────PhISwap(0.25)───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                          │
1: ────PhISwap(0.25)──────PhISwap(0.25)^-0.784──────────────────────────PhISwap(0.25)────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────